In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### This comes from previous notebooks...

In [12]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

### Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process.

For example:

- `learning_rate` (how much to update models parameters at each batch/epoch)
- `batch_size` (number of data samples propagated through the network before updating parameters)
- `epochs` (number of times to iterate over the dataset)

In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### Optimization loop

Each iteration of the optimizatoin loop is called an _epoch_.

- **Train loop**: iterate over the training dataset and try to converge to optimal parameters.
- **Validation/test loop**: iterate over the test dataset to check if model performance is improving.

The _loss function_ measures the degree of dissimilarity between predictions and real values.

- `MSELoss` for regression
- `NLLLoss` for classification
- `CrossEntropyLoss` combines `NLLLoss` and `LogSoftmax`

In [14]:
loss_fn = nn.CrossEntropyLoss()

### Optimizer

Optimization involves adjusting the model parameters to reduce prediction errors in each training step.

Optimization algorithms define how this process is performed (i.e. Stochastic Gradient Descent, ADAM, RMSProp).

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Inside the training loop, optimizatio happens in three steps:

1. `optimizer.zero_grad()` is called to reset the gradients of model parameters
2. backpropagation of the prediction error after calling `loss.backward()`
3. `optimizer.step()` adjust the parameters by the gradients collected in the backward pass

### Training loop

Let's see the training loop:

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train() # set the model to training mode. Important for batch normalization and dropout

    for batch, (X, y) in enumerate(dataloader):
        # PREDICT AND GET LOSS
        pred = model(X)
        loss = loss_fn(pred, y)

        # BACKPROPAGATION
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

And the test loop...

In [ ]:
def test_loop(dataloader, model, loss_fn):
    model.eval() # set to evaluation mode

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0,0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f'Test error: \n Accuracy: {(100*correct) :>0.1f}%, Avg loss: {test_loss:>8f}\n')



Evaluating the model with `torch.no_grad()` ensures that no gradients are computed during test mode. Also serves to reduce unnecessary gradient computations and memory usage.

Now let's train and test:

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 2.312810 [   64/60000]
loss: 2.299899 [ 6464/60000]
loss: 2.275367 [12864/60000]
loss: 2.265247 [19264/60000]
loss: 2.253852 [25664/60000]
loss: 2.220118 [32064/60000]
loss: 2.229505 [38464/60000]
loss: 2.195182 [44864/60000]
loss: 2.195304 [51264/60000]
loss: 2.151009 [57664/60000]
Test error: 
 Accuracy: 43.4%, Avg loss: 2.152643
Epoch 2
-------------------------------
loss: 2.165602 [   64/60000]
loss: 2.156515 [ 6464/60000]
loss: 2.089532 [12864/60000]
loss: 2.107549 [19264/60000]
loss: 2.061419 [25664/60000]
loss: 1.990726 [32064/60000]
loss: 2.016929 [38464/60000]
loss: 1.935583 [44864/60000]
loss: 1.949271 [51264/60000]
loss: 1.859064 [57664/60000]
Test error: 
 Accuracy: 55.2%, Avg loss: 1.867659
Epoch 3
-------------------------------
loss: 1.902026 [   64/60000]
loss: 1.873279 [ 6464/60000]
loss: 1.746281 [12864/60000]
loss: 1.794640 [19264/60000]
loss: 1.697418 [25664/60000]
loss: 1.635470 [32064/60000]
loss: 1.659032 [38464/6000